In [76]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.providers.fake_provider import FakeMelbourneV2, FakeCasablancaV2, FakeVigoV2, FakeLagosV2, FakeGuadalupeV2, FakeGuadalupe, FakeGeneva

#backend = FakeVigoV2() #5 qubits
#backend = FakeCasablancaV2() #7 qubits
#backend = FakeLagosV2() #7 qubits
backend = FakeMelbourneV2() #14 qubits
#backend = FakeGuadalupeV2() #16 qubits
#backend = FakeGeneva() #27 qubits

qubits=[0,1,2,3]
num_qubits = 4

def trotterLayer(h,J,dt,n):
    n=2
    trotterLayer = QuantumCircuit(num_qubits)
    trotterLayer.rx(dt*4*h, qubits)
    trotterLayer.cx(*zip(*[(qubits[2*i], qubits[2*i+1]) for i in range(n)]))
    trotterLayer.rz(-4*J*dt, [qubits[2*i+1] for i in range(n)])
    trotterLayer.cx(*zip(*[(qubits[2*i], qubits[2*i+1]) for i in range(n)]))
    trotterLayer.cx(*zip(*[(qubits[2*i+1], qubits[2*i+2]) for i in range(n-1)]))
    trotterLayer.rz(-4*J*dt, [qubits[2*i+2] for i in range(n-1)])
    trotterLayer.cx(*zip(*[(qubits[2*i+1], qubits[2*i+2]) for i in range(n-1)]))
    return trotterLayer

h = 1
J = -.15
dt = .2
n = 2

def maketrotterCircuit(s):
    tL = trotterLayer(h, J, dt, n)
    trotterCircuit = QuantumCircuit(num_qubits)
    for i in range(s):
        trotterCircuit = trotterCircuit.compose(tL)
        trotterCircuit.barrier()

    trotterCircuit = transpile(trotterCircuit, backend)
    return trotterCircuit

circuits = [maketrotterCircuit(i) for i in range(1,15)]

used_qubits = set()
for circuit in circuits: 
    for c in circuit: #look at the commands
        for bit in c.qubits: #record which qubits they use
            used_qubits.add(bit.index) #and save those
qubits = used_qubits
[4-q for q in qubits]
#circuits[0].draw()

C:\Users\fabri\AppData\Local\Temp\ipykernel_8592\3089960230.py:47: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  used_qubits.add(bit.index) #and save those


[4, 3, 2, 1]

In [74]:
noisyresult = []
shots=10000
for circ in circuits:
    qc = circ.copy()
    qc.measure_all()
    count= backend.run(qc, shots=5000).result().get_counts()
    count = {tuple(int(k) for i, k in enumerate(key) if len(key)-1-i in qubits):count[key] for key in count.keys()}
    tot = 0
    for key in count.keys():
        num = sum([(-1)**bit for bit in key])
        tot += num*count[key]
    noisyresult.append(tot/(1024*n*2))
print(noisyresult)

[3.328125, 0.04052734375, -2.86328125, -3.822265625, -2.408203125, 0.271484375, 2.5986328125, 3.2392578125, 1.9853515625, -0.18359375, -1.8466796875, -2.2275390625, -1.19287109375, 0.349609375]


In [78]:
res = []
shots=10000
for circ in circuits:
    qc = circ.copy()
    qc.measure_all()
    count= Aer.get_backend('qasm_simulator').run(qc, shots=shots).result().get_counts()
    count = {tuple(int(k) for i, k in enumerate(key) if len(key)-1-i in qubits):count[key] for key in count.keys()}
    tot = 0
    for key in sorted(count.keys()):
        num = sum([(-1)**bit for bit in key])
        tot += num*count[key]
    res.append(tot/(shots*1024*n*2))
print(res)

{(0, 0, 0, 1): 940, (0, 1, 0, 1): 170, (1, 1, 1, 0): 39, (0, 0, 1, 0): 942, (0, 0, 0, 0): 5117, (1, 0, 0, 0): 965, (1, 1, 0, 0): 154, (0, 1, 0, 0): 922, (1, 0, 0, 1): 177, (1, 0, 1, 1): 25, (0, 0, 1, 1): 154, (0, 1, 1, 0): 170, (0, 1, 1, 1): 35, (1, 0, 1, 0): 156, (1, 1, 0, 1): 29, (1, 1, 1, 1): 5}
(0, 0, 0, 0) 4 20468
(0, 0, 0, 1) 2 22348
(0, 0, 1, 0) 2 24232
(0, 0, 1, 1) 0 24232
(0, 1, 0, 0) 2 26076
(0, 1, 0, 1) 0 26076
(0, 1, 1, 0) 0 26076
(0, 1, 1, 1) -2 26006
(1, 0, 0, 0) 2 27936
(1, 0, 0, 1) 0 27936
(1, 0, 1, 0) 0 27936
(1, 0, 1, 1) -2 27886
(1, 1, 0, 0) 0 27886
(1, 1, 0, 1) -2 27828
(1, 1, 1, 0) -2 27750
(1, 1, 1, 1) -4 27730
{(0, 0, 1, 0): 586, (1, 1, 0, 0): 673, (1, 1, 1, 0): 676, (0, 1, 0, 1): 580, (1, 0, 1, 0): 613, (0, 1, 1, 1): 646, (1, 1, 1, 1): 701, (1, 1, 0, 1): 672, (0, 1, 1, 0): 627, (1, 0, 1, 1): 629, (1, 0, 0, 1): 595, (0, 0, 1, 1): 614, (1, 0, 0, 0): 608, (0, 1, 0, 0): 611, (0, 0, 0, 0): 584, (0, 0, 0, 1): 585}
(0, 0, 0, 0) 4 2336
(0, 0, 0, 1) 2 3506
(0, 0, 1, 0) 2

In [60]:
a = [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1),
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)]

b =[(0, 0, 0, 0, 0), #4 
(0, 0, 0, 1, 0), #2 
(0, 0, 1, 0, 0), #2 
(0, 0, 1, 1, 0), #0 
(0, 1, 0, 0, 0), #2 
(0, 1, 0, 1, 0), #0 
(0, 1, 1, 0, 0), #0 
(0, 1, 1, 1, 0), #-2
(1, 0, 0, 0, 0)] #2 

for key in a:
    num = sum([(-1)**bit for bit in key[:4]])
    print(num)

15
13
13
11
13
11
11
9
13
